# 自动并行
:label:`sec_auto_para`


深度学习框架（例如，MXNet和PyTorch）在后端自动构建计算图。使用计算图，系统可以了解所有依赖关系，并可以选择性地并行执行多个不相互依赖的任务以提高速度。例如，:numref:`fig_asyncgraph` 在 :numref:`sec_async` 中独立初始化了两个变量。因此系统可以选择并行执行它们。


通常，单个操作符会使用所有CPU上的所有计算资源或单个GPU上的所有计算资源。例如，`dot` 操作符将使用所有CPU上的所有核心（和线程），即使单台机器上有多个CPU处理器也是如此。对于单个GPU也是如此。因此，对于单设备计算机来说，并行化并不是特别有用。对于多设备情况则更为重要。虽然并行化通常在多个GPU之间最为相关，但增加本地CPU会略微提升性能。例如，参见 :citet:`Hadjis.Zhang.Mitliagkas.ea.2016`，该文献专注于结合GPU和CPU训练计算机视觉模型。借助自动并行框架的便利，我们只需几行Python代码即可实现相同的目标。更广泛地说，我们对自动并行计算的讨论集中在使用CPU和GPU进行并行计算，以及计算和通信的并行化。

请注意，运行本节中的实验至少需要两个GPU。

In [1]:
import torch
from d2l import torch as d2l

## 并行计算在GPU上

让我们首先定义一个参考工作负载来进行测试：下面的 `run` 函数在我们选择的设备上执行10次矩阵-矩阵乘法，使用分配到两个变量 `x_gpu1` 和 `x_gpu2` 中的数据。

In [2]:
devices = d2l.try_all_gpus()
def run(x):
    return [x.mm(x) for _ in range(50)]

x_gpu1 = torch.rand(size=(4000, 4000), device=devices[0])
x_gpu2 = torch.rand(size=(4000, 4000), device=devices[1])

现在我们将函数应用于数据。为了确保缓存不会影响结果，我们在测量前先对设备进行预热，即在任一设备上执行一次遍历。`torch.cuda.synchronize()` 会等待 CUDA 设备上所有流中的所有内核完成。它接受一个 `device` 参数，即我们需要同步的设备。如果 `device` 参数为 `None`（默认值），则使用由 `current_device()` 给出的当前设备。

In [3]:
run(x_gpu1)
run(x_gpu2)  # Warm-up all devices
torch.cuda.synchronize(devices[0])
torch.cuda.synchronize(devices[1])

with d2l.Benchmark('GPU1 time'):
    run(x_gpu1)
    torch.cuda.synchronize(devices[0])

with d2l.Benchmark('GPU2 time'):
    run(x_gpu2)
    torch.cuda.synchronize(devices[1])

GPU1 time: 0.4660 sec


GPU2 time: 0.4510 sec


如果我们去掉两个任务之间的`synchronize`语句，系统就可以自动并行化在这两个设备上的计算。

In [4]:
with d2l.Benchmark('GPU1 & GPU2'):
    run(x_gpu1)
    run(x_gpu2)
    torch.cuda.synchronize()

GPU1 & GPU2: 0.4659 sec

在上述情况下，总执行时间少于各部分之和，因为深度学习框架会自动调度在两个GPU设备上的计算，而无需用户编写复杂的代码。

## 并行计算与通信

在很多情况下，我们需要在不同设备之间移动数据，比如在CPU和GPU之间，或在不同的GPU之间。例如，在我们希望执行分布式优化时，需要在多张加速卡上汇总梯度。让我们通过在GPU上进行计算然后将结果复制回CPU来模拟这种情况。

In [5]:
def copy_to_cpu(x, non_blocking=False):
    return [y.to('cpu', non_blocking=non_blocking) for y in x]

with d2l.Benchmark('Run on GPU1'):
    y = run(x_gpu1)
    torch.cuda.synchronize()

with d2l.Benchmark('Copy to CPU'):
    y_cpu = copy_to_cpu(y)
    torch.cuda.synchronize()

Run on GPU1: 0.4656 sec


Copy to CPU: 2.3125 sec


这有些低效。请注意，我们可以在列表的其余部分仍在计算时就开始将`y`的部分内容复制到CPU。这种情况发生在例如我们在小批量上计算（反向传播）梯度时。某些参数的梯度会比其他参数的梯度更早可用。因此，在GPU仍在运行时就开始利用PCI-Express总线带宽对我们有利。在PyTorch中，诸如`to()`和`copy_()`等几个函数接受一个显式的`non_blocking`参数，这允许调用者在不需要同步时绕过同步。设置`non_blocking=True`允许我们模拟这种场景。

In [6]:
with d2l.Benchmark('Run on GPU1 and copy to CPU'):
    y = run(x_gpu1)
    y_cpu = copy_to_cpu(y, True)
    torch.cuda.synchronize()

Run on GPU1 and copy to CPU: 1.6907 sec


两次操作所需的总时间（如预期的那样）少于它们各自所需时间的总和。
请注意，这项任务与并行计算不同，因为它使用了不同的资源：CPU和GPU之间的总线。实际上，我们可以同时在两个设备上进行计算和通信。如上所述，计算和通信之间存在依赖关系：必须先计算`y[i]`才能将其复制到CPU。幸运的是，系统可以在计算`y[i]`的同时复制`y[i-1]`以减少总运行时间。

我们通过一个示例来说明在CPU和两个GPU上训练简单的两层MLP时的计算图及其依赖关系，如:label:`fig_twogpu`所示。手动调度由此产生的并行程序将非常痛苦。这时拥有一个基于图的计算后端来进行优化就显得很有优势了。

![The computational graph and its dependencies of a two-layer MLP on a CPU and two GPUs.](../img/twogpu.svg)
:label:`fig_twogpu`


## 总结

* 现代系统拥有多样化的设备，例如多个GPU和CPU。它们可以并行、异步地使用。
* 现代系统还拥有多样化的通信资源，如PCI Express、存储（通常是固态硬盘或通过网络）、以及网络带宽。为了达到最高效率，这些资源可以并行使用。
* 通过自动并行计算和通信，后端可以提高性能。

## 练习

1. 在本节定义的`run`函数中执行了八个操作。它们之间没有任何依赖关系。设计一个实验来看看深度学习框架是否会自动并行执行它们。
1. 当单个操作符的工作负载足够小的时候，即使是在单个CPU或GPU上，并行化也可以提供帮助。设计一个实验来验证这一点。
1. 设计一个实验，该实验利用CPU、GPU上的并行计算及两者间的通信。
1. 使用调试器如NVIDIA的[Nsight](https://developer.nvidia.com/nsight-compute-2019_5)来验证你的代码是否高效。
1. 设计包含更复杂数据依赖性的计算任务，并运行实验看看你是否可以获得正确的结果同时提高性能。

[讨论](https://discuss.d2l.ai/t/1681)